In [1]:
import os
import torch
import re
import json
import gdown
from datasets import Dataset
import pandas as pd
from peft import LoraConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig, GenerationConfig
from tqdm.auto import tqdm
from trl import DPOTrainer, DPOConfig

c:\Users\Dell\Documents\llm_2024\llama_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.


In [3]:
import inseq

# List of attribution methods to be used
attribution_methods = ['saliency', 'attention']

for method in attribution_methods:
    print(f"======= Attribution Method: {method} =======")
    # Load the Chinese-to-English translation model and set up the attribution method
    model = inseq.load_model("Helsinki-NLP/opus-mt-zh-en", method)

    # Apply attribution to the input text using the specified method
    attribution_result = model.attribute(
        input_texts="我喜歡機器學習和人工智慧。",
        step_scores=["probability"],
    )

    # Remove '▁' from the tokenizer in the prefix to avoid confusion (You can ignore this part of code)
    for attr in attribution_result.sequence_attributions:
        for item in attr.source:
            item.token = item.token.replace('▁', '')
        for item in attr.target:
            item.token = item.token.replace('▁', '')

    # Display the attribution results
    attribution_result.show()

c:\Users\Dell\Documents\llm_2024\llama_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


======= Attribution Method: saliency =======


c:\Users\Dell\Documents\llm_2024\llama_env\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Dell\.cache\huggingface\hub\models--Helsinki-NLP--opus-mt-zh-en. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\Users\Dell\Documents\llm_2024\llama_env\Lib\site-packages\transformers\generation\configuration_u

,I,like,machine,learning,and,artificial,intelligence,.,</s>
我喜歡,0.23,0.438,0.084,0.064,0.183,0.052,0.058,0.25,0.193
機器,0.162,0.135,0.491,0.225,0.13,0.069,0.07,0.141,0.151
學,0.074,0.054,0.151,0.188,0.07,0.03,0.032,0.069,0.061
習,0.07,0.078,0.106,0.267,0.118,0.052,0.055,0.083,0.064
和,0.067,0.047,0.026,0.05,0.158,0.036,0.03,0.073,0.073
人工,0.099,0.052,0.039,0.063,0.104,0.399,0.203,0.091,0.114
智慧,0.113,0.068,0.039,0.073,0.109,0.274,0.437,0.111,0.153
。,0.119,0.066,0.02,0.027,0.083,0.045,0.057,0.118,0.124
</s>,0.066,0.063,0.044,0.043,0.046,0.043,0.058,0.064,0.068
probability,0.767,0.589,0.425,0.671,0.856,0.647,0.866,0.891,0.896


======= Attribution Method: attention =======


ValueError: Step scores are not supported for final step methods since they do not iterate over the full sequence. Please remove the step scores and compute them separatly passing method='dummy'.

In [5]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("gpt2-xl", device_map="auto")

In [6]:
for method in attribution_methods:
    # Load the model with the specified attribution method
    inseq_model = inseq.load_model(model, method)

    # Apply attribution to the input text using the specified method
    attribution_result = inseq_model.attribute(
        input_texts="The first president of America is",
        step_scores=["probability"],
    )

    # Remove 'Ġ' from GPT2's BPE tokenizer in the prefix to avoid confusion (You can ignore this part of code)
    for attr in attribution_result.sequence_attributions:
        for item in attr.source:
            item.token = item.token.replace('Ġ', '')
        for item in attr.target:
            item.token = item.token.replace('Ġ', '')

    # Display the attribution results
    attribution_result.show()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Attributing with saliency...: 100%|██████████| 26/26 [00:24<00:00,  1.25s/it]


,a,man,named,George,Washington,.,He,was,a,man,of,great,integrity,",",and,he,was,a,man,of
The,0.191,0.191,0.189,0.162,0.138,0.252,0.144,0.112,0.15,0.121,0.093,0.088,0.111,0.081,0.056,0.066,0.066,0.081,0.066,0.079
first,0.158,0.122,0.129,0.102,0.077,0.092,0.074,0.069,0.073,0.071,0.068,0.056,0.064,0.044,0.03,0.038,0.043,0.044,0.041,0.041
president,0.256,0.235,0.197,0.199,0.162,0.135,0.14,0.128,0.122,0.122,0.094,0.111,0.118,0.098,0.07,0.073,0.086,0.091,0.076,0.073
of,0.081,0.082,0.076,0.073,0.05,0.058,0.044,0.045,0.047,0.05,0.029,0.043,0.034,0.026,0.029,0.028,0.025,0.025,0.026,0.022
America,0.225,0.18,0.206,0.223,0.127,0.113,0.107,0.096,0.109,0.114,0.062,0.076,0.086,0.067,0.045,0.045,0.044,0.06,0.054,0.055
is,0.089,0.089,0.059,0.047,0.048,0.077,0.053,0.054,0.061,0.049,0.046,0.042,0.044,0.035,0.034,0.032,0.029,0.032,0.03,0.032
a,,0.101,0.03,0.027,0.028,0.031,0.031,0.029,0.031,0.034,0.033,0.026,0.023,0.021,0.029,0.026,0.022,0.019,0.024,0.015
man,,,0.113,0.064,0.054,0.047,0.058,0.06,0.063,0.115,0.068,0.047,0.053,0.046,0.035,0.033,0.039,0.038,0.056,0.034
named,,,,0.102,0.082,0.068,0.091,0.093,0.068,0.062,0.105,0.075,0.059,0.057,0.044,0.057,0.049,0.047,0.047,0.048
George,,,,,0.234,0.05,0.083,0.083,0.068,0.065,0.058,0.069,0.066,0.057,0.041,0.052,0.051,0.047,0.044,0.05


ValueError: Step scores are not supported for final step methods since they do not iterate over the full sequence. Please remove the step scores and compute them separatly passing method='dummy'.